# Simplified AES Plaintext Recovery (FNN)
In this experiment, the network tries to guess the plaintext from the ciphertext, helped with ascii per-byte correction. 

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from dataset.datasets import SimplifiedAESDatasetCiphertextPlaintext
from pipeline import *

2023-05-11 16:42:49.400173: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Importing the dataset

In [4]:
data = SimplifiedAESDatasetCiphertextPlaintext('small')

train_labels, train_samples, test_labels, test_samples = data.get_data()

In [5]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (1516543, 16)
===== Label Shape: (16,)
===== Training Samples Shape: (1516543, 16)
===== Sample Shape: (16,)
===== Testing Labels Shape: (649947, 16)
===== Testing Samples Shape: (649947, 16)


In [57]:
train_labels = train_labels[:500]
train_samples = train_samples[:500]

## Creating the model

In [58]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

Add customization here.

In [59]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = len(train_labels[0])
print("Output dimension: {}".format(dim))

# units per hidden layer
units = dim*16
print("Units per hidden layer: {}".format(units))

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
# 0.1 to 0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=1000,
    decay_rate=0.01)
optimizer = Adam(learning_rate=0.001)
metrics = ['accuracy', 'binary_accuracy']
epochs = 1000
batch_size = 100

Output dimension: 16
Units per hidden layer: 256


### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [60]:
# Type of model
neural_network = Sequential()

# Input layer
neural_network.add(Input(shape=input_shape))

# Hidden layers
#neural_network.add(BatchNormalization())
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))
neural_network.add(Dense(units=units, activation='relu'))

# Output layer
neural_network.add(Dense(units=dim, activation='sigmoid'))

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_mse, metrics=metrics)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 256)               4352      
                                                                 
 dense_31 (Dense)            (None, 256)               65792     
                                                                 
 dense_32 (Dense)            (None, 256)               65792     
                                                                 
 dense_33 (Dense)            (None, 256)               65792     
                                                                 
 dense_34 (Dense)            (None, 16)                4112      
                                                                 
Total params: 205,840
Trainable params: 205,840
Non-trainable params: 0
_________________________________________________________________


### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [61]:
history = train_model(neural_network, train_samples, train_labels, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/1000
5/5 [==============================] - 1s 46ms/step - loss: 0.2320 - accuracy: 0.1444 - binary_accuracy: 0.6683 - val_loss: 0.1964 - val_accuracy: 0.2600 - val_binary_accuracy: 0.7362
Epoch 2/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.1808 - accuracy: 0.2133 - binary_accuracy: 0.7403 - val_loss: 0.1699 - val_accuracy: 0.2600 - val_binary_accuracy: 0.7538
Epoch 3/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.1668 - accuracy: 0.2133 - binary_accuracy: 0.7550 - val_loss: 0.1568 - val_accuracy: 0.2600 - val_binary_accuracy: 0.7900
Epoch 4/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.1555 - accuracy: 0.1022 - binary_accuracy: 0.7836 - val_loss: 0.1473 - val_accuracy: 0.0000e+00 - val_binary_accuracy: 0.7987
Epoch 5/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.1443 - accuracy: 0.0000e+00 - binary_accuracy: 0.7942 - val_loss: 0.1367 - val_accuracy: 0.0000e+00 - val_binary_accuracy: 0.8000
Epo

In [67]:
print(history.history['loss'])
print(history.history['val_loss'])
print(history.history['val_accuracy'])

[0.2320386916399002, 0.18079587817192078, 0.16680674254894257, 0.15546691417694092, 0.14429835975170135, 0.13375793397426605, 0.12314662337303162, 0.11337495595216751, 0.10415776073932648, 0.09636140614748001, 0.08985955268144608, 0.08383307605981827, 0.07905638962984085, 0.07339588552713394, 0.0684218481183052, 0.06477036327123642, 0.05957719683647156, 0.055506594479084015, 0.0517854318022728, 0.047369834035634995, 0.04424929618835449, 0.04152318462729454, 0.03869403153657913, 0.03693920373916626, 0.034351855516433716, 0.031724829226732254, 0.029562581330537796, 0.027305420488119125, 0.024921929463744164, 0.023350486531853676, 0.021556388586759567, 0.020302096381783485, 0.019039414823055267, 0.017653847113251686, 0.016641756519675255, 0.015626797452569008, 0.014932402409613132, 0.014097281731665134, 0.01309002935886383, 0.012253175489604473, 0.011526775546371937, 0.01074199564754963, 0.00998299103230238, 0.009639191441237926, 0.009098876267671585, 0.008606280200183392, 0.0081936614587

In [63]:
training_loss = [0.0096, 0.0013, 0.0014, 0.00096996, 0.00095061, 0.00076971, 0.0002997, 0.0006391, 0.0005801, 0.00048286]
validation_loss = [0.0012, 0.0011, 0.00074558, 0.00071942, 0.00068666, 0.00050246, 0.00049399, 0.00049060, 0.0004091, 0.00040874]
validation_accuracy = [0.2627, 0.3425, 0.2618, 0.4119, 0.4112, 0.6048, 0.8779, 0.7888, 0.7932, 0.8410]

### Testing
Here, we evaluate the neural network with the test data.

This block stays the same.

In [64]:
results = neural_network.evaluate(test_samples, test_labels, batch_size=batch_size)
print("Test loss: {}".format(results[0]))
print("Test accuracy: {}".format(results[1]))

6500/6500 [==============================] - 11s 2ms/step - loss: 0.0527 - accuracy: 0.4944 - binary_accuracy: 0.9400
Test loss: 0.05265556275844574
Test accuracy: 0.49443262815475464


### Plaintext Recovery in action

In [65]:
size = 1000

In [66]:
predictions = [predict_sample(neural_network, test_samples[i]) for i in range(1000)]

1/1 [==============================] - 0s 19ms/step


In [68]:
metrics = [get_metrics((predictions[i], test_labels[i])) for i in range(1000)]

In [69]:
correct_bytes = 0
correct_predictions = 0
for m in metrics:
    correct_bytes += m[0]
    correct_predictions += m[1]
                             
print("Correct bytes: {}".format(correct_bytes))
print("Byte accuracy: {}".format(correct_bytes/(2*1000)))
print("Correct predictions: {}".format(correct_predictions))
print("Prediction accuracy: {}".format(correct_predictions/1000))

Correct bytes: 1487
Byte accuracy: 0.7435
Correct predictions: 681
Prediction accuracy: 0.681


In [70]:
for i in range(1000):
    print("prediction: " + prediction_to_string(test_labels[i]) + " || " + prediction_to_string(predictions[i]))

prediction: do || dm
prediction:  t ||  t
prediction: he || he
prediction: m  || m 
prediction: no || no
prediction: t. || t.
prediction: 

 || 


prediction: 33 || ne
prediction: :3 || re
prediction: 3  || 3 
prediction: An || An
prediction: d  || d 
prediction: wh || wh
prediction: en || en
prediction:  t ||  t
prediction: hi || hi
prediction: s  || s 
prediction: co || ai
prediction: me || me
prediction: th || th
prediction:  t ||  t
prediction: o  || n)
prediction: pa ||  e
prediction: ss || ss
prediction: ,  || , 
prediction: (l || 
a
prediction: o, || nl
prediction:  i ||  i
prediction: t  || t 
prediction: wi || wi
prediction: ll || ll
prediction:  c ||  c
prediction: om || om
prediction: e, || cl
prediction: )  || 8(
prediction: th || th
prediction: en || en
prediction:  s ||  s
prediction: ha || ha
prediction: ll || ll
prediction: 
t || Ce
prediction: he || he
prediction: y  || x`
prediction: kn || kn
prediction: ow || Og
prediction:  t ||  t
prediction: ha || ha
prediction: t

In [71]:
save_model(neural_network, "s_aes_feed_forward_small")